In [ ]:
from corelab.core.session import CoreLabSession

session = CoreLabSession('sagemaker-distribution', 'draft', aws_profile='sagemaker-role')
session.retrieve_image('3.3')
# image_uris.retrieve(
#             framework='datascience',
#             region='eu-central-1',
#             version='foo',
#             py_version="py3",  # only for some frameworks
#             instance_type='ml.m5.large',
#             sagemaker_session=self.core_session
#         )

In [ ]:
import boto3
import re
import os
os.environ['AWS_PROFILE'] = 'machiel-crystalline'
def find_python310_images(region='eu-central-1'):
    """Find all training images with Python 3.10+"""

    ecr = boto3.client('ecr', region_name=region)
    registry_id = '545423591354'  # AWS Deep Learning Containers

    try:
        # List all repositories
        repos = ecr.describe_repositories(
            registryId=registry_id,
            maxResults=100
        )

        py310_images = []

        for repo in repos['repositories']:
            repo_name = repo['repositoryName']

            # Only look at training images
            if 'training' not in repo_name:
                continue

            # Get image tags
            try:
                images = ecr.list_images(
                    registryId=registry_id,
                    repositoryName=repo_name,
                    maxResults=100
                )

                for image in images.get('imageIds', []):
                    tag = image.get('imageTag', '')
                    # Look for py310, py311, py312 tags
                    if re.search(r'py31[0-2]', tag) and 'cpu' in tag:
                        full_uri = f"{registry_id}.dkr.ecr.{region}.amazonaws.com/{repo_name}:{tag}"
                        py310_images.append({
                            'framework': repo_name.replace('-training', ''),
                            'tag': tag,
                            'uri': full_uri
                        })

            except Exception:
                continue

        # Sort and display
        py310_images.sort(key=lambda x: (x['framework'], x['tag']))

        print("Images with Python 3.10+:")
        for img in py310_images[:20]:  # Show first 20
            print(f"\n{img['framework']}:")
            print(f"  Tag: {img['tag']}")
            print(f"  URI: {img['uri']}")

        return py310_images

    except Exception as e:
        print(f"Error: {e}")
        return []

# Run discovery
images = find_python310_images()